In [1]:
import keras.backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
from keras.datasets import mnist

In [3]:
from keras.layers import Input
from keras.layers import Dense

from keras.layers import Lambda

from keras.models import Model

In [4]:
def z_samp(z_statis):#z_mean , z_log_var):
    batch_size = K.shape(z_statis[0])[0]
    data_dims = K.shape(z_statis[0])[1]
    
    std_gaussian = K.random_normal([batch_size , data_dims])
    
    return z_mean + K.exp(0.5*z_log_var) * std_gaussian

In [5]:
(x_train , y_train) , (x_test , y_test) = mnist.load_data()

In [6]:
x_train = np.reshape(x_train , (-1 , 784))
x_train = x_train/255
x_test = np.reshape(x_test , (-1 , 784))
x_test = x_test/255

In [7]:
###
#encoder

In [8]:
inputs = Input(shape=(784,))
x = Dense(512 , activation = 'relu')(inputs)

#不指定激活函数 就是使用线性函数 a(x)=x
z_mean = Dense(2)(x)
z_log_var = Dense(2)(x)

In [9]:
z = Lambda(z_samp , output_shape=(2,))([z_mean , z_log_var])

In [10]:
encoder = Model(inputs , outputs=[z_mean , z_log_var , z])

In [10]:
encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512)          401920      input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 2)            1026        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 2)            1026        dense_1[0][0]                    
__________________________________________________________________________________________________
lambda_1 (

In [41]:
from keras.utils import plot_model

In [12]:
plot_model(encoder , to_file='model.png' , show_shapes=True)

In [21]:
###
#decoder

In [11]:
decoder_input = Input((2,))
x = Dense(512 , activation='relu')(decoder_input)

decoder_output = Dense(784 , activation='sigmoid')(x)

In [12]:
decoder = Model(inputs = decoder_input , outputs = decoder_output)

In [17]:
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               1536      
_________________________________________________________________
dense_5 (Dense)              (None, 784)               402192    
Total params: 403,728
Trainable params: 403,728
Non-trainable params: 0
_________________________________________________________________


In [ ]:
###
#VAE

In [13]:
outputs = decoder(encoder(inputs)[2]) #decoder的输入就是encoder输出的第三个 （也就是第2个 从0开始）

In [14]:
VAE = Model(inputs=inputs , outputs=outputs)

In [20]:
VAE.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
model_1 (Model)              [(None, 2), (None, 2), (N 403972    
_________________________________________________________________
model_2 (Model)              (None, 784)               403728    
Total params: 807,700
Trainable params: 807,700
Non-trainable params: 0
_________________________________________________________________


In [21]:
plot_model(VAE , to_file='VAE.png' , show_shapes=True)

In [15]:
from keras.losses import mse
from keras.losses import binary_crossentropy

In [17]:
'''
losses = mse(inputs , outputs)
#losses = binary_crossentropy(inputs , outputs)

losses*=784

k1_loss = 1+z_log_var - K.square(z_mean) - K.exp(z_log_var)
k1_loss = K.sum(k1_loss , axis=-1)

k1_loss *= -0.5

VAE_loss = K.mean(losses + k1_loss)
'''

In [18]:
def VAE_loss(x , x_hat):
    losses = mse(inputs , outputs) #MSE
    #losses = binary_crossentropy(inputs , outputs) #crossentropy
    losses *= 784
    
    k1_loss = 1+z_log_var - K.square(z_mean) - K.exp(z_log_var)
    k1_loss = K.sum(k1_loss , axis=-1)

    k1_loss *= -0.5
    
    return K.mean(losses + k1_loss)

In [66]:
VAE.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 784)               0         
_________________________________________________________________
model_4 (Model)              [(None, 2), (None, 2), (N 403972    
_________________________________________________________________
model_5 (Model)              (None, 784)               403728    
Total params: 807,700
Trainable params: 807,700
Non-trainable params: 0
_________________________________________________________________


In [19]:
VAE.compile(optimizer='adam' , loss=VAE_loss)

In [22]:
VAE.fit(x_train[:4] ,x_train[:4], batch_size=2 , epochs=200)

Epoch 1/200
4/4 [==============================] - 0s 13ms/step - loss: 16.3096
Epoch 2/200
4/4 [==============================] - 0s 9ms/step - loss: 17.2666
Epoch 3/200
4/4 [==============================] - 0s 10ms/step - loss: 18.7991
Epoch 4/200
4/4 [==============================] - 0s 10ms/step - loss: 16.5686
Epoch 5/200
4/4 [==============================] - 0s 18ms/step - loss: 15.7374
Epoch 6/200
4/4 [==============================] - 0s 15ms/step - loss: 15.2993
Epoch 7/200
4/4 [==============================] - 0s 13ms/step - loss: 14.9246
Epoch 8/200
4/4 [========================